In [ ]:
import cv2

img = cv2.imread("../input/landuse-scene-classification/images_train_test_val/test/beach/beach_000022.png")

img.shape

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img)

In [ ]:
train_dir = "../input/landuse-scene-classification/images_train_test_val/train"
test_dir = "../input/landuse-scene-classification/images_train_test_val/test"
val_dir = "../input/landuse-scene-classification/images_train_test_val/validation"

In [ ]:
# processing the data 

import tensorflow as tf

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.)

val_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.)


In [ ]:
# creating datasets

train_data = train_gen.flow_from_directory(train_dir,
                                           target_size = (224,224),
                                           class_mode = "categorical",
                                           batch_size = 32
                                           )

test_data = test_gen.flow_from_directory(test_dir,
                                         target_size = (224,224),
                                         class_mode = "categorical",
                                         batch_size = 32
                                         )
val_data = val_gen.flow_from_directory(val_dir,
                                       target_size = (224,224),
                                       class_mode = "categorical",
                                       batch_size = 32
                                       )

In [ ]:
image, label = train_data.next()
print(image.shape)
print(label.shape)

In [ ]:
class_label = list(train_data.class_indices.keys())


In [ ]:
# visualizing the train dataset

import matplotlib.pyplot as plt
import tensorflow as tf

plt.figure(figsize=(16,12))

image, label = train_data.next()
for i in range(18):
  plt.subplot(6,3,i+1)
  plt.imshow(image[i])
  plt.title(class_label[tf.argmax(label[i])])

  plt.axis("off")


In [ ]:
# building a CNN model

import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential([
                             layers.Conv2D(filters= 32, kernel_size= 2, activation="relu", input_shape=(224,224,3)),
                             layers.MaxPooling2D(pool_size= 2),

                             layers.Conv2D(filters = 32, kernel_size= 2, activation="relu"),
                             layers.MaxPooling2D(pool_size= 2),

                             layers.Conv2D(filters= 32, kernel_size= 2, activation= "relu"),
                             layers.MaxPooling2D(pool_size=2),

                             layers.Conv2D(filters= 32, kernel_size= 2, activation= "relu"),
                             layers.MaxPooling2D(pool_size=2),

                            
                             layers.Conv2D(filters= 32, kernel_size= 2, activation= "relu"),
                             layers.Conv2D(filters= 32, kernel_size= 2, activation= "relu"),
                             layers.Conv2D(filters= 32, kernel_size= 2, activation= "relu"),

                             layers.Flatten(),

                             layers.Dense(256, activation="relu"),

                             layers.Dense(21, activation="softmax")


])

In [ ]:
# compiling the model

model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

In [ ]:
# fitting data to the model

model.fit(train_data,
          epochs = 5,
          steps_per_epoch = len(train_data),
          validation_data = val_data,
          validation_steps = len(val_data)
          )

In [ ]:
model_evaluation = model.evaluate(test_data)

In [ ]:
print(f"Model Accuarcy: {model_evaluation[1] * 100: 0.2f} %")

In [ ]:
#visualizing the prediction
import matplotlib.pyplot as plt
import tensorflow as tf

plt.figure(figsize=(16,12))

image, label = test_data.next()
for i in range(18):
  model_pred = model.predict(image)
  plt.subplot(6,3,i+1)
  plt.imshow(image[i])
  plt.title(f"Prediction: {class_label[tf.argmax(model_pred[i])]}\nOriginal: {class_label[tf.argmax(label[i])]}")

  plt.axis("off")


In [ ]:
# processing the data for efficientnet


train_eff = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                     label_mode = "categorical",
                                                                     image_size = (224,224),
                                                                     batch_size = 32
                                                                    
                                                                     )

test_eff = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                   label_mode = "categorical",
                                                                   image_size = (224,224),
                                                                   batch_size = 32
                                                                   
                                                                   )
val_eff = tf.keras.preprocessing.image_dataset_from_directory(val_dir,
                                                                   label_mode = "categorical",
                                                                   image_size = (224,224),
                                                                   batch_size = 32
                                                                   
                                                                   )

In [ ]:
# building model for efficientnet50
import tensorflow as tf

eff_base = tf.keras.applications.EfficientNetB5(include_top= False)

eff_base.trainable = False

In [ ]:
#buildinb resnet model
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(224,224,3))

x = eff_base(inputs)

x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(21, activation="softmax")(x)

eff_model = tf.keras.Model(inputs, outputs)

In [ ]:
# compiling the model

eff_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

In [ ]:
# fitting data to the model

eff_model.fit( train_eff,
             epochs = 5,
             steps_per_epoch = len(train_eff),
             validation_data = val_eff,
             validation_steps = len(val_eff)
             )

In [ ]:
# tunning the model

eff_base.trainable = True

for layer in eff_base.layers[:-3]:
  eff_base.trainable = False

eff_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

eff_model.fit(train_eff,
              epochs = 5,
              steps_per_epoch = len(train_eff),
              validation_data = val_eff,
              validation_steps = len(val_eff))

In [ ]:
print(f"Efficient Accuracy {eff_model.evaluate(val_eff)[1] * 100 : 0.2f} %")

In [ ]:
labels_name = test_eff.class_names
len(labels_name)

In [ ]:
# predicting and visualizing the images

import tensorflow as tf
import matplotlib.pyplot as plt

plt.figure(figsize = (16,16))

for image,label in test_eff.take(1):
  model_prediction = eff_model.predict(image)

  for i in range(18):
    plt.subplot(6,3,i+1)
    plt.imshow(image[i].numpy().astype("uint8"))
    plt.title(f"Model Prediction : {labels_name[tf.argmax(tf.round(model_prediction[i]))]}\nOriginal: {labels_name[tf.argmax(label[i])]}")
    plt.axis("off")